In [12]:
# Import các thư viện cần thiết
from sentence_transformers import SentenceTransformer
import json
from pathlib import Path
import chromadb

# Load mô hình AI. 
# Việc này có thể mất một chút thời gian ở lần chạy đầu tiên trong session mới.
print("Đang tải mô hình Sentence Transformer...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Tải mô hình thành công!")

Đang tải mô hình Sentence Transformer...
Tải mô hình thành công!


In [2]:
# Path.cwd() sẽ lấy thư mục làm việc hiện tại của notebook
# Nếu notebook đang ở trong /notebooks, chúng ta cần đi ra một cấp (dùng .parent)
# để về thư mục gốc của dự án.
project_root = Path.cwd().parent 

# Bây giờ, xây dựng đường dẫn đến file data từ thư mục gốc
data_path = project_root / "data" / "patterns_final.json"

# Đọc và load dữ liệu từ file JSON
print(f"Đang đọc dữ liệu từ {data_path}...")
with open(data_path, 'r', encoding='utf-8') as f:
    patterns_data = json.load(f)

print(f"Đọc thành công {len(patterns_data)} patterns.")

# In ra thử tên của pattern đầu tiên để kiểm tra
if patterns_data:
    print(f"Tên pattern đầu tiên: {patterns_data[0]['name']}")

Đang đọc dữ liệu từ D:\DU_AN_TOT_NGHIEP\pattern-pedia-ai\data\patterns_final.json...
Đọc thành công 22 patterns.
Tên pattern đầu tiên: Proxy


In [3]:
# Lấy ra danh sách các đoạn văn bản cần mã hóa
# Đây là "nguyên liệu" chính cho AI
texts_to_encode = [pattern['text_for_embedding'] for pattern in patterns_data]

# In ra một vài mẫu để kiểm tra
print("Trích xuất thành công các đoạn text cần mã hóa.")
print("Đây là ví dụ về một đoạn text:")
print("-" * 20)
print(texts_to_encode[0][:300] + "...") # In 300 ký tự đầu

Trích xuất thành công các đoạn text cần mã hóa.
Đây là ví dụ về một đoạn text:
--------------------
Pattern Name: Proxy. Category: Structural patterns. Intent: Intent Proxy is a structural design pattern that lets you provide a substitute or placeholder for another object. A proxy controls access to the original object, allowing you to perform something either before or after the request gets thro...


In [4]:
print("Bắt đầu quá trình mã hóa toàn bộ dữ liệu... Việc này có thể mất một chút thời gian.")

# Dùng model.encode() để biến tất cả các đoạn text thành vector
# show_progress_bar=True để hiển thị thanh tiến trình
embeddings = model.encode(texts_to_encode, show_progress_bar=True)

print("Mã hóa hoàn tất!")

Bắt đầu quá trình mã hóa toàn bộ dữ liệu... Việc này có thể mất một chút thời gian.


Batches: 100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.26s/it]

Mã hóa hoàn tất!


In [5]:
# Kiểm tra kết quả
# embeddings là một numpy array
print(f"Dạng của kết quả: {type(embeddings)}")
print(f"Kích thước của array (số lượng pattern, số chiều của vector): {embeddings.shape}")

# In ra 5 chiều đầu tiên của vector đầu tiên để xem thử
print(f"5 chiều đầu tiên của vector đầu tiên: {embeddings[0][:5]}")

Dạng của kết quả: <class 'numpy.ndarray'>
Kích thước của array (số lượng pattern, số chiều của vector): (22, 384)
5 chiều đầu tiên của vector đầu tiên: [-0.06451211 -0.01585064  0.02820135 -0.07741234 -0.1043975 ]


In [6]:
# Hàm trợ giúp để tìm index của một pattern bằng tên
def get_pattern_index(pattern_name):
    for i, pattern in enumerate(patterns_data):
        if pattern['name'].lower() == pattern_name.lower():
            return i
    return -1 # Trả về -1 nếu không tìm thấy

# --- CHỌN CÁC CẶP ĐỂ KIỂM TRA ---

# Cặp 1: Rất liên quan (cùng là Creational, cùng giải quyết việc tạo đối tượng)
idx_factory = get_pattern_index("Factory Method")
idx_abstract_factory = get_pattern_index("Abstract Factory")

# Cặp 2: Không liên quan (một cái về tạo đối tượng, một cái về hành vi)
idx_singleton = get_pattern_index("Singleton")
idx_observer = get_pattern_index("Observer")

# In ra để xác nhận đã tìm đúng index
print(f"Index của Factory Method: {idx_factory}")
print(f"Index của Abstract Factory: {idx_abstract_factory}")
print(f"Index của Singleton: {idx_singleton}")
print(f"Index của Observer: {idx_observer}")

Index của Factory Method: 11
Index của Abstract Factory: 10
Index của Singleton: 7
Index của Observer: 16


In [7]:
# Lấy ra các vector embedding từ array
embedding_factory = embeddings[idx_factory]
embedding_abstract_factory = embeddings[idx_abstract_factory]
embedding_singleton = embeddings[idx_singleton]
embedding_observer = embeddings[idx_observer]

print("Đã lấy thành công các vector để kiểm tra.")

Đã lấy thành công các vector để kiểm tra.


In [10]:
# Import thư viện cần thiết để tính độ tương đồng
from sentence_transformers.util import cos_sim

# Tính độ tương đồng cho cặp liên quan
similarity_related = cos_sim(embedding_factory, embedding_abstract_factory)

# Tính độ tương đồng cho cặp không liên quan
similarity_unrelated = cos_sim(embedding_singleton, embedding_observer)

print("--- KẾT QUẢ KIỂM TRA CHẤT LƯỢNG EMBEDDING ---")
print(f"Độ tương đồng giữa 'Factory Method' và 'Abstract Factory' (liên quan): {similarity_related.item():.4f}")
print(f"Độ tương đồng giữa 'Singleton' và 'Observer' (không liên quan): {similarity_unrelated.item():.4f}")

--- KẾT QUẢ KIỂM TRA CHẤT LƯỢNG EMBEDDING ---
Độ tương đồng giữa 'Factory Method' và 'Abstract Factory' (liên quan): 0.7809
Độ tương đồng giữa 'Singleton' và 'Observer' (không liên quan): 0.4973


In [13]:
# Tạo một client sẽ lưu dữ liệu vào thư mục 'db' trong thư mục gốc của dự án
# (ChromaDB sẽ tự tạo thư mục này nếu chưa có)
client = chromadb.PersistentClient(path="../db") 

# In ra để kiểm tra xem client đã được tạo chưa
print(client)

In [14]:
# Tạo hoặc lấy ra một collection có tên là 'test_collection'
collection = client.get_or_create_collection(name="test_collection")

# In ra để kiểm tra
print(collection)

Collection(name=test_collection)


In [15]:
collection.add(
    documents=[
        "This is a document about dogs.",
        "This is a document about cats and dogs.",
        "The weather is nice today."
    ],
    metadatas=[
        {'source': 'animal_facts'}, 
        {'source': 'animal_facts'},
        {'source': 'daily_news'}
    ],
    ids=["doc1", "doc2", "doc3"] # ID phải là duy nhất
)

print("Đã thêm 3 documents vào collection.")

C:\Users\Admin\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████| 79.3M/79.3M [00:25<00:00, 3.20MiB/s]


Đã thêm 3 documents vào collection.


In [16]:
# Truy vấn collection với một câu hỏi
results = collection.query(
    query_texts=["What are some furry pets?"],
    n_results=2 # Yêu cầu trả về 2 kết quả gần nhất
)

# In kết quả ra một cách đẹp mắt
import pprint
pprint.pprint(results)

{'data': None,
 'distances': [[1.014564037322998, 1.0898027420043945]],
 'documents': [['This is a document about cats and dogs.',
                'This is a document about dogs.']],
 'embeddings': None,
 'ids': [['doc2', 'doc1']],
 'included': ['metadatas', 'documents', 'distances'],
 'metadatas': [[{'source': 'animal_facts'}, {'source': 'animal_facts'}]],
 'uris': None}
